In [1]:
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [7]:
data = np.load('face_data.npy')
print(data[:5])
np.random.shuffle(data)
print(data[:5])
X = data[:, 1:]
Y = data[:,0]

[['Gurprasad' '202' '202' ... '94' '93' '91']
 ['Gurprasad' '42' '42' ... '205' '204' '204']
 ['Gurprasad' '196' '196' ... '26' '26' '27']
 ['Gurprasad' '219' '218' ... '35' '37' '39']
 ['Gurprasad' '201' '202' ... '121' '123' '120']]
[['Gurprasad' '201' '202' ... '121' '123' '120']
 ['Gurprasad' '204' '204' ... '65' '63' '66']
 ['Harshneet' '199' '201' ... '111' '110' '109']
 ['Harshneet' '200' '200' ... '97' '100' '101']
 ['Gurprasad' '203' '203' ... '83' '89' '94']]


In [8]:
X = X.reshape(X.shape[0], 300, 300)

In [9]:
X.shape

(20, 300, 300)

In [10]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
print(y)

[0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 0]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [11]:
y.shape

(20, 2)

In [12]:
X, y = X.astype(np.float), y.astype(np.uint8)

In [13]:
X = (X-127.5)/127.5

In [18]:
max(X[0][0])

0.7647058823529411

In [19]:
in_layer = Input(shape = (300,300))
reshape = Reshape((300, 300, 1))(in_layer)
c1 = Conv2D(32, (3,3))(reshape)
p1 = MaxPooling2D((2, 2))(c1)
c2 = Conv2D(64, (2,2))(p1)
p2 = MaxPooling2D((2, 2))(c2)
drp1 = Dropout(0.2)(p2)
flat = Flatten()(drp1)
d1 = Dense(512, activation="tanh")(flat)
d2 = Dense(300, activation="tanh")(d1)
drp2 = Dropout(0.2)(d2)
d3 = Dense(100, activation="tanh")(drp2)
drp3 = Dropout(0.1)(d3)
out = Dense(2, activation="softmax")(drp3)

In [20]:
model = Model(in_layer, out)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300)]        0         
_________________________________________________________________
reshape (Reshape)            (None, 300, 300, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 298, 298, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 148, 148, 64)      8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 64)        0     

In [22]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [23]:
model.fit(X, y, epochs=10, steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 44s 4s/step - loss: 1.6456 - accuracy: 0.5500
Epoch 2/10
10/10 [==============================] - 41s 4s/step - loss: 0.7262 - accuracy: 0.4500
Epoch 3/10
10/10 [==============================] - 32s 3s/step - loss: 0.7521 - accuracy: 0.4000
Epoch 4/10
10/10 [==============================] - 30s 3s/step - loss: 1.0270 - accuracy: 0.3000
Epoch 5/10
10/10 [==============================] - 30s 3s/step - loss: 0.7403 - accuracy: 0.7000
Epoch 6/10
10/10 [==============================] - 30s 3s/step - loss: 0.7452 - accuracy: 0.4500
Epoch 7/10
10/10 [==============================] - 30s 3s/step - loss: 0.8079 - accuracy: 0.3500
Epoch 8/10
10/10 [==============================] - 30s 3s/step - loss: 0.7594 - accuracy: 0.4000
Epoch 9/10
10/10 [==============================] - 29s 3s/step - loss: 0.8315 - accuracy: 0.5000
Epoch 10/10
10/10 [==============================] - 34s 3s/step - loss: 0.7248 - accuracy: 0.4500


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
path = "/content/gdrive/My Drive/facemodel.hdf5"
model.save(path, save_format="hdf5")